In [ ]:
%load_ext autoreload
%autoreload 2

In [245]:
import gymnasium as gym
from gymnasium import spaces
import jax
import jax.numpy as jnp

import numpy as np

from os import path
# from scipy.stats import triang
from functools import partial
# import matplotlib.pyplot as plt
import os
# from klax import triangular, make_unsafe_spaces, contained_in_any



In [258]:
from rl_environments import Vandelpol
env = Vandelpol()
state = env.reset()

rng = jax.random.PRNGKey(2025)
rngs = jax.random.split(rng, 5)
samples = jnp.array([env.init_space.sample() for _ in range(5)])
print(samples)


[[-0.19489497  0.4512635 ]
 [ 0.11745233  0.2637241 ]
 [ 0.07182688  0.31471875]
 [ 0.01762978  0.50003624]
 [ 0.14580959  0.47152242]]


/home/wuhao/micromamba/envs/nc/lib/python3.10/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [260]:
s_next = jnp.expand_dims(env.v_next(samples,rngs), axis=1)
print(s_next)

[[[-0.28514767  0.40968606]]

 [[ 0.06470752  0.2539832 ]]

 [[ 0.00888313  0.29425624]]

 [[-0.08237747  0.4758975 ]]

 [[ 0.0515051   0.45784405]]]


In [247]:

def func(a):
    return a[0]

samples = jnp.array([env.init_space.sample() for _ in range(5)])
print(samples)
vfunc = jax.vmap(func)
result = vfunc(samples)
print(result)

[[-0.1097627   0.31561857]
 [ 0.03964382  0.50550896]
 [-0.08534091  0.23951039]
 [ 0.19517188  0.34240845]
 [-0.17276113  0.5143665 ]]
[-0.1097627   0.03964382 -0.08534091  0.19517188 -0.17276113]


In [275]:
dims = 2
grid, steps = [], []
for i in range(dims):
    samples, step = np.linspace(
        -1,
        1,
        4,
        endpoint=False,
        retstep=True,
    )
    grid.append(samples)
    steps.append(step)
# print(grid)

grid = np.meshgrid(*grid)
print(grid)
grid_lb = [x.flatten() for x in grid]
grid_ub = [grid_lb[i] + steps[i] for i in range(dims)]
grid_centers = [grid_lb[i] + steps[i] / 2 for i in range(dims)]
print(grid_lb)
print(grid_ub)
# grid_lb = np.stack(grid_lb, axis=1)
# grid_ub = np.stack(grid_ub, axis=1)
# grid_centers = np.stack(grid_centers, axis=1)

[array([-1. , -0.5,  0. ,  0.5, -1. , -0.5,  0. ,  0.5, -1. , -0.5,  0. ,
        0.5, -1. , -0.5,  0. ,  0.5]), array([-1. , -1. , -1. , -1. , -0.5, -0.5, -0.5, -0.5,  0. ,  0. ,  0. ,
        0. ,  0.5,  0.5,  0.5,  0.5])]
[array([-0.5,  0. ,  0.5,  1. , -0.5,  0. ,  0.5,  1. , -0.5,  0. ,  0.5,
        1. , -0.5,  0. ,  0.5,  1. ]), array([-0.5, -0.5, -0.5, -0.5,  0. ,  0. ,  0. ,  0. ,  0.5,  0.5,  0.5,
        0.5,  1. ,  1. ,  1. ,  1. ])]


In [8]:
# Define individual spaces
space1 = spaces.Box(low=0, high=1, shape=(3,), dtype=float)  # Continuous space with shape (3,)
space2 = spaces.Discrete(5)  # Discrete space with 5 possible values
space3 = spaces.MultiBinary(4)  # Multi-binary space with shape (4,)

# Combine spaces using spaces.Tuple
composite_space = spaces.Tuple((space1, space2, space3))

# Example usage
print("Composite space:", composite_space)
print("Sample from composite space:", composite_space.sample())

Composite space: Tuple(Box(0.0, 1.0, (3,), float64), Discrete(5), MultiBinary(4))
Sample from composite space: (array([0.12005234, 0.33023144, 0.47497074]), 2, array([0, 1, 0, 0], dtype=int8))
